In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [2]:
#Understanding how gating works
num_experts = 4
top_k = 2
n_embed=32

In [3]:
# fake multihead attention output
mh_output = torch.randn(2, 4, n_embed) # (B, T, C) = (2, 4, 32) = (batch_size, block_size, n_embed)
mh_output.shape

torch.Size([2, 4, 32])

In [4]:
topkgate_linear = nn.Linear(n_embed, num_experts)
logits = topkgate_linear(mh_output)
logits.shape

torch.Size([2, 4, 4])

In [5]:
logits

tensor([[[ 0.1266,  0.3873, -0.4022, -0.5019],
         [-1.4505,  0.4356, -0.5226, -0.9259],
         [-0.0429, -0.1706, -0.1338, -0.3950],
         [-0.2503,  0.3080, -0.1050, -0.5048]],

        [[-0.3012,  0.7607, -1.3323,  0.7659],
         [-0.9557,  0.1939, -0.0320, -1.3697],
         [-0.2010, -0.5835,  0.3432, -0.2241],
         [-1.1562,  0.3388, -0.6097,  0.2768]]], grad_fn=<ViewBackward0>)

In [6]:
# Lấy top_k giá trị lớn nhất và chỉ số của chúng từ logits theo chiều cuối cùng (dim=-1).
top_k_logit, top_k_idx = logits.topk(top_k, dim=-1)


In [7]:
top_k_logit

tensor([[[ 0.3873,  0.1266],
         [ 0.4356, -0.5226],
         [-0.0429, -0.1338],
         [ 0.3080, -0.1050]],

        [[ 0.7659,  0.7607],
         [ 0.1939, -0.0320],
         [ 0.3432, -0.2010],
         [ 0.3388,  0.2768]]], grad_fn=<TopkBackward0>)

In [8]:
top_k_idx

tensor([[[1, 0],
         [1, 2],
         [0, 2],
         [1, 2]],

        [[3, 1],
         [1, 2],
         [2, 0],
         [1, 3]]])

Lấy đầu ra sparse gating bằng cách chỉ giữ lại k giá trị hàng đầu tại chỉ số tương ứng của chúng dọc theo chiều cuối cùng. Điền phần còn lại bằng ‘-inf’ và truyền qua hàm kích hoạt softmax. Điều này đẩy các giá trị ‘-inf’ về không, làm cho hai giá trị hàng đầu được nhấn mạnh hơn và tổng bằng 1. Tổng bằng 1 này giúp ích cho việc tính trọng số của đầu ra expert.

In [9]:
# Tạo một tensor zeros có cùng shape với logits (tức (2, 4, num_experts)).
# Các phần tử được điền giá trị âm vô cực (-inf).
# Mục đích là để tạo một mask mà chỉ giữ lại các giá trị top-k.
zeros = torch.full_like(logits, float('-inf'))
zeros

tensor([[[-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf]],

        [[-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf]]])

In [10]:
# Tạo tensor thưa bằng cách đặt các giá trị top_k_logit vào vị trí được chỉ định bởi top_k_idx.
# scatter thay thế các giá trị -inf tại các chỉ số top_k_idx bằng top_k_logit.
# Kết quả sparse_logits có shape (2, 4, num_experts), trong đó:
# Chỉ có top_k giá trị khác -inf ở mỗi chuỗi
# Các vị trí khác vẫn là -inf
sparse_logits = zeros.scatter(-1, top_k_idx, top_k_logit)
sparse_logits

tensor([[[ 0.1266,  0.3873,    -inf,    -inf],
         [   -inf,  0.4356, -0.5226,    -inf],
         [-0.0429,    -inf, -0.1338,    -inf],
         [   -inf,  0.3080, -0.1050,    -inf]],

        [[   -inf,  0.7607,    -inf,  0.7659],
         [   -inf,  0.1939, -0.0320,    -inf],
         [-0.2010,    -inf,  0.3432,    -inf],
         [   -inf,  0.3388,    -inf,  0.2768]]], grad_fn=<ScatterBackward0>)

In [11]:
gating_output= F.softmax(sparse_logits, dim=-1)
gating_output


tensor([[[0.4352, 0.5648, 0.0000, 0.0000],
         [0.0000, 0.7228, 0.2772, 0.0000],
         [0.5227, 0.0000, 0.4773, 0.0000],
         [0.0000, 0.6018, 0.3982, 0.0000]],

        [[0.0000, 0.4987, 0.0000, 0.5013],
         [0.0000, 0.5562, 0.4438, 0.0000],
         [0.3672, 0.0000, 0.6328, 0.0000],
         [0.0000, 0.5155, 0.0000, 0.4845]]], grad_fn=<SoftmaxBackward0>)

Nếu chỉ dùng top-k sẽ rất có khả năng có những expert được chọn nhiều lần, điều đó dẫn đến không cân bằng tải.

=> Phương pháp: Noisy top-k Gating for load balancing

In [ ]:
# First define the top k router module 
class TopkRouter(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        super(TopkRouter, self).__init__()
        self.top_k = top_k
        self.linear =nn.Linear(n_embed, num_experts)
    
    def forward(self, mh_ouput):
        # mh_ouput is the output tensor from multihead self attention block
        logits = self.linear(mh_output)
        top_k_logits, indices = logits.topk(self.top_k, dim=-1) 
        zeros = torch.full_like(logits, float('-inf'))
        #scatter thay thế các giá trị -inf tại các chỉ số top_k_idx bằng top_k_logit.
        sparse_logits = zeros.scatter(-1, indices, top_k_logits)
        router_output = F.softmax(sparse_logits, dim=-1)
        return router_output, indices


In [13]:
#Testing this out:
num_experts = 4
top_k = 2
n_embd = 32

mh_output = torch.randn(2, 4, n_embd)  # Example input
top_k_gate = TopkRouter(n_embd, num_experts, top_k)
gating_output, indices = top_k_gate(mh_output)
gating_output.shape, gating_output, indices

(torch.Size([2, 4, 4]),
 tensor([[[0.0000, 0.0000, 0.5394, 0.4606],
          [0.0000, 0.8703, 0.0000, 0.1297],
          [0.0000, 0.6878, 0.3122, 0.0000],
          [0.6251, 0.0000, 0.3749, 0.0000]],
 
         [[0.0000, 0.7266, 0.0000, 0.2734],
          [0.4494, 0.0000, 0.5506, 0.0000],
          [0.2100, 0.0000, 0.7900, 0.0000],
          [0.7678, 0.2322, 0.0000, 0.0000]]], grad_fn=<SoftmaxBackward0>),
 tensor([[[2, 3],
          [1, 3],
          [1, 2],
          [0, 2]],
 
         [[1, 3],
          [2, 0],
          [2, 0],
          [0, 1]]]))

In [15]:
#Changing the above to accomodate noisy top-k gating
class NoisyTopkRouter(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        super(NoisyTopkRouter, self).__init__()
        self.top_k = top_k
        #layer for router logits
        self.topkroute_linear = nn.Linear(n_embed, num_experts)
        self.noise_linear =nn.Linear(n_embed, num_experts)

    
    def forward(self, mh_output):
        # mh_ouput is the output tensor from multihead self attention block
        logits = self.topkroute_linear(mh_output)

        #Noise logits
        noise_logits = self.noise_linear(mh_output)

        #Adding scaled unit gaussian noise to the logits
        noise = torch.randn_like(logits)*F.softplus(noise_logits)
        noisy_logits = logits + noise

        top_k_logits, indices = noisy_logits.topk(self.top_k, dim=-1)
        zeros = torch.full_like(noisy_logits, float('-inf'))
        sparse_logits = zeros.scatter(-1, indices, top_k_logits)
        router_output = F.softmax(sparse_logits, dim=-1)
        return router_output, indices


In [16]:
#Testing this out, again:
num_experts = 8
top_k = 2
n_embd = 16

mh_output = torch.randn(2, 4, n_embd)  # Example input
noisy_top_k_gate = NoisyTopkRouter(n_embd, num_experts, top_k)
gating_output, indices = noisy_top_k_gate(mh_output)
gating_output.shape, gating_output, indices
#It works!!


(torch.Size([2, 4, 8]),
 tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.4747, 0.0000, 0.5253, 0.0000],
          [0.0000, 0.4845, 0.5155, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.9715, 0.0000, 0.0000, 0.0000, 0.0000, 0.0285, 0.0000],
          [0.4486, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5514]],
 
         [[0.0000, 0.0000, 0.0000, 0.4555, 0.0000, 0.0000, 0.5445, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.5230, 0.0000, 0.4770, 0.0000],
          [0.2455, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7545],
          [0.0000, 0.0000, 0.5009, 0.0000, 0.0000, 0.0000, 0.4991, 0.0000]]],
        grad_fn=<SoftmaxBackward0>),
 tensor([[[6, 4],
          [2, 1],
          [1, 6],
          [7, 0]],
 
         [[6, 3],
          [4, 6],
          [7, 0],
          [2, 6]]]))

In [17]:
# Creating a sparse Mixture of Experts module

class SparseMoE(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        super(SparseMoE, self).__init__()
        self.router = NoisyTopkRouter(n_embed, num_experts, top_k)
        self.experts = nn.ModuleList([Expert(n_embed) for _ in range(num_experts)])
        self.top_k = top_k

    def forward(self, x):
        gating_output, indices = self.router(x)
        final_output = torch.zeros_like(x)

        # Reshape inputs for batch processing
        flat_x = x.view(-1, x.size(-1))
        flat_gating_output = gating_output.view(-1, gating_output.size(-1))

        # Process each expert in parallel
        for i, expert in enumerate(self.experts):
            # Create a mask for the inputs where the current expert is in top-k
            expert_mask = (indices == i).any(dim=-1)
            flat_mask = expert_mask.view(-1)

            if flat_mask.any():
                expert_input = flat_x[flat_mask]
                expert_output = expert(expert_input)

                # Extract and apply gating scores
                gating_scores = flat_gating_output[flat_mask, i].unsqueeze(1)
                weighted_output = expert_output * gating_scores

                # Update final output additively by indexing and adding
                final_output[expert_mask] += weighted_output.squeeze(1)

        return final_output



In [18]:
import torch
import torch.nn as nn

#Let's test this out
num_experts = 8
top_k = 2
n_embd = 16
dropout=0.1

mh_output = torch.randn(4, 8, n_embd)  # Example multi-head attention output
sparse_moe = SparseMoE(n_embd, num_experts, top_k)
final_output = sparse_moe(mh_output)
print("Shape of the final output:", final_output.shape)


NameError: name 'Expert' is not defined